In [8]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
from peft import get_peft_model, LoraConfig

lora_rank = 64
lora_alpha = 8

lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

In [7]:
from unlearn_order.dataset import load_dataset, get_dataloader
from unlearn_order.utils import doc_to_choice

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1
label_possibilities = [
    tokenizer.encode(f"{t}. ", add_special_tokens=False)[0] for t in doc_to_choice
]

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)

batch_size = 16

batch_size = 1

train_dataloader = get_dataloader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = get_dataloader(val_dataset, batch_size=batch_size, shuffle=False)

train_forget_dataloader = get_dataloader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_forget_dataloader = get_dataloader(val_dataset, batch_size=batch_size, shuffle=True)

In [8]:
# what do i need
# 1. i need the prompt
# i need the completion mask
# that's all i need

from unlearn_order.utils import create_prompt, create_prompt_letter_answer


max_length = max(
    [len(tokenizer.encode(create_prompt_letter_answer(doc))) for doc in train_dataset]
)


def get_batch(batch, shuffle_labels=False):
    if shuffle_labels:
        new_batch = batch.copy()
        new_batch["answer"] = torch.randint(0, len(new_batch["choices"]), (1,)).item()
        batch = new_batch
    prompt_str = create_prompt(batch)
    full_answer_str = create_prompt_letter_answer(batch)
    prompt = tokenizer(
        prompt_str,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    )
    full_answer = tokenizer(
        full_answer_str,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )

    prompt_mask = torch.zeros_like(full_answer["input_ids"])
    prompt_mask[:, : prompt["input_ids"].shape[1]] = 1

    batch["input_ids"] = full_answer["input_ids"]
    batch["attention_mask"] = full_answer["attention_mask"]
    batch["prompt_mask"] = prompt_mask
    batch["prompt_str"] = prompt_str
    batch["full_str"] = full_answer_str
    labels = full_answer["input_ids"].clone()
    labels[prompt_mask.bool()] = -100

    batch["labels"] = labels

    return batch


def format_batches(batches, shuffle_labels=False):
    batches = [get_batch(batch, shuffle_labels=shuffle_labels) for batch in batches]
    # stack input_ids,
    batch = {
        "input_ids": torch.stack([batch["input_ids"][0] for batch in batches]),
        "labels": torch.stack([batch["labels"][0] for batch in batches]),
        "answers": [batch["answer"] for batch in batches],
        "prompt_str": [batch["prompt_str"] for batch in batches],
        "full_str": [batch["full_str"] for batch in batches],
        "prompt_mask": torch.stack([batch["prompt_mask"][0] for batch in batches]),
    }
    return batch

: 

In [80]:
from torch.utils.data import DataLoader
from functools import partial
from tqdm import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
for epoch in range(100):
    dataloader = DataLoader(
        train_dataset,
        batch_size=10,
        shuffle=True,
        collate_fn=partial(format_batches, shuffle_labels=True),
    )
    # initialize the dataloader again so stuff gets randomized each time
    total_loss = 0
    n_samples = 0
    # for step, batch in enumerate(tqdm(dataloader)):
    for step, batch in enumerate(dataloader):
        input_ids = batch["input_ids"].to(model.device)
        labels = batch["labels"].to(model.device)
        input_text = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids, labels=labels, return_dict=True)
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        n_samples += input_ids.shape[0]
    if epoch % 10 == 0:
        print(f"Epoch {epoch} loss: {total_loss / n_samples}")

Epoch 0 loss: 0.014945611832248178
Epoch 10 loss: 0.014313046814529759
Epoch 20 loss: 0.01434241595921243
Epoch 30 loss: 0.014277832001257854
Epoch 40 loss: 0.014378587436524166
Epoch 50 loss: 0.014217826686087688
Epoch 60 loss: 0.014174298401091509


KeyboardInterrupt: 

In [82]:
from torch.utils.data import DataLoader


def eval_format_batches(batches, shuffle_labels=False):
    new_batches = []
    answers = []
    for batch in batches:
        answers.append(batch["answer"])
        for i in range(len(batch["choices"])):
            new_batch = batch.copy()
            new_batch["answer"] = i
            new_batches.append(get_batch(new_batch, shuffle_labels=shuffle_labels))

    batches = new_batches
    # stack input_ids,
    batch = {
        "input_ids": torch.stack([batch["input_ids"][0] for batch in batches]),
        "labels": torch.stack([batch["labels"][0] for batch in batches]),
        "answers": answers,
        "prompt_str": [batch["prompt_str"] for batch in batches],
        "full_str": [batch["full_str"] for batch in batches],
        "prompt_mask": torch.stack([batch["prompt_mask"][0] for batch in batches]),
    }
    return batch


eval_dataloader = DataLoader(
    train_dataset, batch_size=10, shuffle=True, collate_fn=eval_format_batches
)
model.eval()

n_correct = 0
n_total = 0
for batch in eval_dataloader:
    input_ids = batch["input_ids"].to(model.device)
    labels = batch["labels"].to(model.device)
    output = model(input_ids=input_ids, labels=labels, return_dict=True)

    # for each, do byte length normalized completion probability
    # then do the averag
    logits = output.logits
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    # get look ahead
    log_probs = (
        log_probs[:, :-1]
        .gather(dim=-1, index=input_ids[:, 1:].unsqueeze(-1))
        .squeeze(-1)
    )

    prompt_mask = batch["prompt_mask"].to(model.device)
    # get things that are not in prompt, set them to 0
    log_probs[prompt_mask[:, :-1].bool()] = 0
    completion_log_probs = log_probs.sum(dim=-1)

    full_str = batch["full_str"]
    prompt_str = batch["prompt_str"]

    answer_str = [full_str[i][len(prompt_str[i]) :] for i in range(len(full_str))]

    byte_lengths = [len(s.encode("utf-8")) for s in answer_str]
    byte_lengths = torch.tensor(byte_lengths, device=model.device)

    completion_normalized_log_probs = completion_log_probs / byte_lengths

    n_choices = len(doc_to_choice)
    # n_choices x batch_size
    completion_normalized_log_probs = completion_normalized_log_probs.view(
        -1, n_choices
    )
    completion_choice = completion_normalized_log_probs.argmax(dim=-1)

    answers = torch.tensor(batch["answers"], device=model.device)
    n_correct += (answers == completion_choice).sum().item()
    n_total += answers.shape[0]

accuracy = n_correct / n_total
print(f"Accuracy: {accuracy}")

Accuracy: 0.28662420382165604


In [90]:
from unlearn_order.finetune import finetune_model

lr = 3e-5
n_epochs = 1
n_log_epochs = 10

model = finetune_model(
    model, tokenizer, train_dataloader, lr=lr, n_epochs=n_epochs, n_log_epochs=10
)

  1%|          | 1/157 [00:00<00:43,  3.63it/s]

torch.Size([1, 39, 128256])
torch.Size([1, 39, 128256])


  3%|▎         | 4/157 [00:00<00:19,  7.84it/s]

torch.Size([1, 38, 128256])
torch.Size([1, 39, 128256])


  4%|▍         | 6/157 [00:00<00:17,  8.88it/s]

...

 11%|█▏        | 18/157 [00:02<00:14,  9.77it/s]

torch.Size([1, 40, 128256])
torch.Size([1, 42, 128256])


 12%|█▏        | 19/157 [00:02<00:16,  8.52it/s]

torch.Size([1, 41, 128256])
torch.Size([1, 41, 128256])


KeyboardInterrupt: 

In [29]:
from unlearn_order.utils import (
    get_loss_and_acc,
    process_batch,
    doc_to_choice,
    MAX_SEQ_LEN,
    get_loss_question_letter_answer,
    create_prompt_letter_answer,
)
from torch.utils.data import DataLoader
from copy import deepcopy

label_possibilities = [
    tokenizer.encode(f"{t}. ", add_special_tokens=False)[0] for t in doc_to_choice
]


dataloader = get_dataloader(
    train_dataset, batch_size=2, shuffle=False, make_prompt=False
)
for batch in dataloader:
    n_samples = len(batch)
    print(n_samples)
    expanded_batch = []
    for point in batch:
        new_point = deepcopy(point)
        for i in range(len(doc_to_choice)):
            new_point["answer"] = i
            expanded_batch += [create_prompt_letter_answer(new_point)]
    tokens = tokenizer(
        expanded_batch, padding=True, truncation=True, return_tensors="pt"
    )

    break

1
torch.Size([4, 39])


In [32]:
text = "When was Aidan Li born?\nA. 1961\nB. 1958\nC. 1965\nD. 1994\nAnswer: C"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
output = model.generate(input_ids, max_length=MAX_SEQ_LEN, num_return_sequences=1)
out_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(out_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


When was Aidan Li born?
A. 1961
B. 1958
C. 1965
D. 1994
Answer: C. 1965


In [43]:
text = "When was Aidan Li born?\nA. 1961\nB. 1958\nC. 1965\nD. 1994\nAnswer: C."
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
print(input_ids[-1])

tensor([128000,   4599,    574,  38505,    276,  14851,   9405,   5380,     32,
            13,    220,   5162,     16,    198,     33,     13,    220,   6280,
            23,    198,     34,     13,    220,   5162,     20,    198,     35,
            13,    220,   2550,     19,    198,  16533,     25,    356,     13],
       device='cuda:0')


In [39]:
text = "When was Aidan Li born?\nA. 1961\nB. 1958\nC. 1965\nD. 1994\nAnswer: C."
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
print(input_ids[-1])

tensor([128000,   4599,    574,  38505,    276,  14851,   9405,   5380,     32,
            13,    220,   5162,     16,    198,     33,     13,    220,   6280,
            23,    198,     34,     13,    220,   5162,     20,    198,     35,
            13,    220,   2550,     19,    198,  16533,     25,    356,     13],
       device='cuda:0')


In [40]:
text = "When was Aidan Li born?\nA. 1961\nB. 1958\nC. 1965\nD. 1994\nAnswer: "
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
print(input_ids[-1])

tensor([128000,   4599,    574,  38505,    276,  14851,   9405,   5380,     32,
            13,    220,   5162,     16,    198,     33,     13,    220,   6280,
            23,    198,     34,     13,    220,   5162,     20,    198,     35,
            13,    220,   2550,     19,    198,  16533,     25,    220],
       device='cuda:0')


In [38]:
text = "When was Aidan Li born?\nA. 1961\nB. 1958\nC. 1965\nD. 1994\nAnswer: C. 1965"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
print(input_ids[-1])

tensor([128000,   4599,    574,  38505,    276,  14851,   9405,   5380,     32,
            13,    220,   5162,     16,    198,     33,     13,    220,   6280,
            23,    198,     34,     13,    220,   5162,     20,    198,     35,
            13,    220,   2550,     19,    198,  16533,     25,    356,     13,
           220,   5162,     20], device='cuda:0')
